In [3]:
# Only do this install once, for experimenting with hugging face's transformers
# CPU usage, just to ensure that model architecture will be working.
# Will later likely need to use GPUs using a virtual environment,
# IF: our dataset is too big, possibly not the case.
#pip install transformers[torch]

# Same here, run once
#!pip install datasets

   ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
   ---------------------- ---------------- 317.4/542.0 kB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 542.0/542.0 kB 8.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/371.6 kB ? eta -:--:--
   -------------------------------- ------ 307.2/371.6 kB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 371.6/371.6 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   - -------------------------------------- 0.7/25.9 MB 22.4 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - 

In [1]:
# imports need necessary installs found above

from naiveModel import NBLangIDModel
from BERTModel import BERTGenreClassification, train_model
import pandas as pd
from sklearn.model_selection import train_test_split
from util import get_dataloader

# don't need cuda until using virtual machine
from torch import manual_seed#, cuda

c:\Users\oscar\anaconda3\envs\ml-0451\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
naiveBayes = NBLangIDModel()

In [6]:
# initialize model

# change later, if using virtual machine
device ='cpu' #"cuda" if cuda.is_available() else "cpu"

# seed the model before initializing weights so that your code is deterministic
manual_seed(457)

freeze_bert = False # change later
batch_size = 16 # default
epochs = 5 # default
learning_rate = 1e-2 # default

model = BERTGenreClassification(freeze_bert = freeze_bert).to(device)

In [7]:
# load data, train test split
descriptions = pd.read_csv("cleanedData.csv")

print("Shape before dropping NaN values:", descriptions.shape)
descriptions = descriptions.dropna()
print("Shape after dropping NaN values:", descriptions.shape)

#TESTING THIS
# print("Rows to be dropped:")
# print(descriptions[descriptions['description'].isna() | ~descriptions['description'].apply(lambda x: isinstance(x, str))])
# #drop the 511 rows where description column is NaN or not a string
# descriptions = descriptions.dropna(subset=['description'])
# descriptions = descriptions[descriptions['description'].apply(lambda x: isinstance(x, str))]

train, test = train_test_split(descriptions, test_size=0.2)

Shape before dropping NaN values: (42661, 5)
Shape after dropping NaN values: (37478, 5)


In [8]:
train.drop("Unnamed: 0", axis= 1)
test.drop("Unnamed: 0", axis= 1)

,description,genre1,genre2,genre3
3808,One night changed my life forever.Beau Bennett...,'New Adult','Romance','Contemporary'
10260,The greatest future histories in science ficti...,'Science Fiction','Fiction','Science Fiction Fantasy'
22566,"""This fine translation is a god-send. . . . Su...",'Science','Nonfiction','History'
42357,"At the head of The Colbert Report, one of the ...",'Philosophy','Nonfiction','Humor'
27844,Jaybee Corbell awoke after more than 200 years...,'Science Fiction','Fiction','Time Travel'
...,...,...,...,...
19319,Josh Turner wants Hannah more than life itself...,'New Adult','Romance','Contemporary Romance'
4131,José Saramago is a master at pacing. Readers u...,'Fiction','Literature','Portugal'
38095,ZANDER — The most feared of the Eternal Guardi...,'Paranormal Romance','Paranormal','Romance'
42631,In today's world no self-respecting English-la...,'Nonfiction','Humor','Language'


In [96]:
train_X = train['description']
train_y1 = train['genre1']
train_y2 = train['genre2']
train_y3 = train['genre3']

In [10]:
# fit the NB model 
naiveBayes.fit(train_X.tolist(), train_y1.tolist())

In [57]:
train['text'] = pd.Series()
train['label'] = None

In [58]:
label_vocab = naiveBayes.labels
print(label_vocab)

label_as_id = {l:k  for k, l in enumerate(label_vocab)}

{"'Linguistics'", "'Dutch Literature'", "'Israel'", "'Jewish'", "'Theosophy'", "'Nonfiction'", "'Education'", "'True Crime'", "'Metaphysics'", "'French Literature'", "'Magical Realism'", "'Language'", "'Middle Grade'", "'Realistic Fiction'", "'War'", "'21st Century'", "'New Adult'", "'Beer'", "'Lesbian'", "'Alchemy'", "'Spirituality'", "'Science'", "'International Relations'", "'Ireland'", "'Tarot'", "'Amish'", "'Sports'", "'Humor'", "'Italian Literature'", "'New Testament'", "'Mental Illness'", "'Fitness'", "'Historical Fiction'", "'Atheism'", "'Light Novel'", "'Programming'", "'Sex Work'", "'China'", "'Dogs'", "'Noir'", "'Espionage'", "'Witches'", "'Dark'", "'Adoption'", "'Vampires'", "'Japan'", "'Medical'", "'Geography'", "'Relationships'", "'Irish Literature'", "'Contemporary Romance'", "'Christian'", "'Fae'", "'Amazon'", "'Boarding School'", "'Physics'", "'Romantic Suspense'", "'Food'", "'Philosophy'", "'Novels'", "'Germany'", "'Teen'", "'Quilting'", "'Pirates'", "'Disability'", "

In [84]:
bert_train, bert_val = train_test_split(train, test_size=0.1)
bert_train.drop("Unnamed: 0", axis= 1)
bert_val.drop("Unnamed: 0", axis= 1)

,description,genre1,genre2,genre3,text,label
864,Luce would die for Daniel.And she has. Over an...,'Young Adult','Fantasy','Angels',NaN,None
26147,The thrilling follow-up to the groundbreaking ...,'Young Adult','Science Fiction','Dystopia',NaN,None
12013,It's been eighteen months since the Raxter Sch...,'Horror','Young Adult','LGBT',NaN,None
2515,Dark schemes unfold--and Sophie's loyalty is p...,'Fantasy','Middle Grade','Young Adult',NaN,None
7572,From the beloved author of Treasure Island Ori...,'Classics','Historical Fiction','Fiction',NaN,None
...,...,...,...,...,...,...
2744,"Disguised as a boy, Alanna of Trebond becomes ...",'Fantasy','Young Adult','Fiction',NaN,None
29391,You get to the point where you can just say it...,'Romance','Contemporary Romance','Contemporary',NaN,None
4622,Jason and Rachel were not born in Lyrian. They...,'Fantasy','Young Adult','Middle Grade',NaN,None
7456,In the Bone there is a house.In the house ther...,'Thriller','New Adult','Dark',NaN,None


In [99]:
from util import BERT_preprocess
#bert_train = BERT_preprocess(bert_train, label_as_id)
bert_val['genre1'].str.strip()
bert_val['genre2'].str.strip()
bert_val['genre3'].str.strip()

bert_val = BERT_preprocess(bert_val, label_as_id)

'Young Adult'
'Young Adult'
'Horror'
'Fantasy'
'Classics'
'Science Fiction'
'M M Romance'
'Nonfiction'
'Science Fiction'
'Horror'
'Mystery'
'Finance'
'Childrens'
'Realistic Fiction'
'Young Adult'
'Fantasy'
'Nonfiction'
'Doctor Who'
'Memoir'
'Music'
'Animals'
'Romance'
'Fiction'
'Fantasy'
'Fantasy'
'Classics'
'Erotica'
'Young Adult'
'Fiction'
'Design'
'Fiction'
'Science Fiction'
'History'
'Art'
'Fiction'
'Fantasy'
'Fiction'
'Fantasy'
'Nonfiction'
'Nonfiction'
'Paranormal Romance'
'Nonfiction'
'Nonfiction'
'Buddhism'
'Economics'
'Fantasy'
'Angels'
'Manga'
'Romance'
'Mystery'
'Paranormal Romance'
'Fantasy'
'Romantic Suspense'
'Nonfiction'
'Nonfiction'
'Nonfiction'
'Historical Fiction'
'Picture Books'
'Fantasy'
'Romance'
'Christian Fiction'
'Historical Fiction'
'Fantasy'
'Mystery'
'Paranormal'
'Vampires'
'Fantasy'
'Urban Fantasy'
'Fiction'
'Vampires'
'Star Wars'
'Thriller'
'Historical Fiction'
'Romance'
'History'
'Fashion'
'Memoir'
'Romance'
'Art'
'Science Fiction'
'Poetry'
'Paranormal'
'F

In [95]:
bert_val.head()
print(sum(bert_val.iloc[10, 6]))

print(label_as_id)
print(bert_val.iloc[10, 4])

0
{"'Linguistics'": 0, "'Dutch Literature'": 1, "'Israel'": 2, "'Jewish'": 3, "'Theosophy'": 4, "'Nonfiction'": 5, "'Education'": 6, "'True Crime'": 7, "'Metaphysics'": 8, "'French Literature'": 9, "'Magical Realism'": 10, "'Language'": 11, "'Middle Grade'": 12, "'Realistic Fiction'": 13, "'War'": 14, "'21st Century'": 15, "'New Adult'": 16, "'Beer'": 17, "'Lesbian'": 18, "'Alchemy'": 19, "'Spirituality'": 20, "'Science'": 21, "'International Relations'": 22, "'Ireland'": 23, "'Tarot'": 24, "'Amish'": 25, "'Sports'": 26, "'Humor'": 27, "'Italian Literature'": 28, "'New Testament'": 29, "'Mental Illness'": 30, "'Fitness'": 31, "'Historical Fiction'": 32, "'Atheism'": 33, "'Light Novel'": 34, "'Programming'": 35, "'Sex Work'": 36, "'China'": 37, "'Dogs'": 38, "'Noir'": 39, "'Espionage'": 40, "'Witches'": 41, "'Dark'": 42, "'Adoption'": 43, "'Vampires'": 44, "'Japan'": 45, "'Medical'": 46, "'Geography'": 47, "'Relationships'": 48, "'Irish Literature'": 49, "'Contemporary Romance'": 50, "'

In [14]:
# BERT data loaders and training
from util import BERT_preprocess
bert_train = BERT_preprocess(bert_train, label_as_id)
bert_val = BERT_preprocess(bert_val, label_as_id)

# have to change these to implement from df, not from csv
train_dataloader = get_dataloader(bert_train, batch_size=batch_size)
val_dataloader = get_dataloader(bert_val, batch_size=batch_size)


# currently, embedding dimensions are wrong
#train_model(model, train_dataloader, dev_dataloader, epochs, learning_rate)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().